# When you have a bipolar pair that is CA1 or DG, how often is each electrode in each region? Or white matter?

In [5]:
import pandas as pd; pd.set_option('display.max_columns', 30)
import numpy as np
from cmlreaders import CMLReader, get_data_index
from ptsa.data.filters import ButterworthFilter
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from copy import copy
from scipy import stats
import pickle
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/Downloads/ptsa_plot-master/')
sys.path.append('/home1/john/johnModules')
from brain_labels import HPC_labels, ENT_labels, PHC_labels, temporal_lobe_labels,\
                         MFG_labels, IFG_labels, nonHPC_MTL_labels
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *

df = get_data_index("r1") # all RAM subjects
exp = 'catFR1' # 'FR1' 'catFR1' 'RepFR1'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# 575 FR sessions. first 18 of don't load so skip those 
exp_df = df[df.experiment==exp]
if exp == 'FR1':
    exp_df = exp_df[
                    ((df.subject!='R1015J') | (df.session!=0)) & 
                    ((df.subject!='R1063C') | (df.session!=1)) & 
                    ((df.subject!='R1093J') | (~df.session.isin([1,2]))) &
                    ((df.subject!='R1100D') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1120E') | (df.session!=0)) &
                    ((df.subject!='R1122E') | (df.session!=2)) &
                    ((df.subject!='R1154D') | (df.session!=0)) &
                    ((df.subject!='R1186P') | (df.session!=0)) &
                    ((df.subject!='R1201P') | (~df.session.isin([0,1]))) &
                    ((df.subject!='R1216E') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1277J') | (df.session!=0)) &
                    ((df.subject!='R1413D') | (df.session!=0)) & 
                    ((df.subject!='R1123C') | (df.session!=2)) & # artifacts that bleed through channels (see SWR FR1 prob sessions ppt)
                    ((df.subject!='R1151E') | (~df.session.isin([1,2]))) & # more bleed-through artifacts (see same ppt)
                    ((df.subject!='R1275D') | (df.session!=3))  # 3rd session an actual repeat of 2nd session (Paul should have removed from database by now)
    #                 (df.subject!='R1065J') # sub with 9000 trials
                   ] 
elif exp == 'catFR1': 
    exp_df = exp_df[
                    ((df.subject!='R1044J') | (df.session!=0)) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1491T') | (~df.session.isin([1,3,5]))) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1486J') | (~df.session.isin([4,5,6,7]))) & # repeated data...will be removed at some point... @@
                    ((df.subject!='R1501J') | (~df.session.isin([0,1,2,3,4,5]))) & # these weren't catFR1 (and they don't load right anyway)
                    ((df.subject!='R1235E') | (df.session!=0)) & # split EEG filenames error...documented on Asana
                    ((df.subject!='R1310J') | (df.session!=1)) & # session 1 is just a repeat of session 0
                    ((df.subject!='R1239E') | (df.session!=0)) # some correlated noise (can see in catFR1 problem sessions ppt)
    ]
elif exp == 'RepFR1':
    exp_df = exp_df[
                    (df.subject!='R1564J') # clearly something wrong with these EEG when looking at ripple raster
                    ]
exp_df.shape

# exp_df = exp_df[540:] # for FR1 this is R1385E-onwwards

exp_df

/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


(514, 17)

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
24,NaN,protocols/r1/subjects/R1004D/experiments/catFR...,protocols/r1/subjects/R1004D/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1004D/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1004D/localizations/0/m...,NaN,0,R1004D,R1004D,NaN,protocols/r1/subjects/R1004D/experiments/catFR...
45,NaN,protocols/r1/subjects/R1013E/experiments/catFR...,protocols/r1/subjects/R1013E/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1013E/experiments/catFR...,1,NaN,0,protocols/r1/subjects/R1013E/localizations/0/m...,NaN,0,R1013E,R1013E_1,NaN,protocols/r1/subjects/R1013E/experiments/catFR...
58,NaN,protocols/r1/subjects/R1015J/experiments/catFR...,protocols/r1/subjects/R1015J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1015J/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1015J/localizations/0/m...,NaN,0,R1015J,R1015J,NaN,protocols/r1/subjects/R1015J/experiments/catFR...
64,NaN,protocols/r1/subjects/R1016M/experiments/catFR...,protocols/r1/subjects/R1016M/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1016M/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1016M/localizations/0/m...,NaN,0,R1016M,R1016M,NaN,protocols/r1/subjects/R1016M/experiments/catFR...
65,NaN,protocols/r1/subjects/R1016M/experiments/catFR...,protocols/r1/subjects/R1016M/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1016M/experiments/catFR...,0,NaN,1,protocols/r1/subjects/R1016M/localizations/0/m...,NaN,1,R1016M,R1016M,NaN,protocols/r1/subjects/R1016M/experiments/catFR...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2624,NaN,protocols/r1/subjects/R1617S/experiments/catFR...,protocols/r1/subjects/R1617S/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1617S/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1617S/localizations/0/m...,NaN,3,R1617S,R1617S,3.4,protocols/r1/subjects/R1617S/experiments/catFR...
2625,NaN,protocols/r1/subjects/R1617S/experiments/catFR...,protocols/r1/subjects/R1617S/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1617S/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1617S/localizations/0/m...,NaN,4,R1617S,R1617S,3.4,protocols/r1/subjects/R1617S/experiments/catFR...
2626,NaN,protocols/r1/subjects/R1617S/experiments/catFR...,protocols/r1/subjects/R1617S/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1617S/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1617S/localizations/0/m...,NaN,5,R1617S,R1617S,3.4,protocols/r1/subjects/R1617S/experiments/catFR...
2627,NaN,protocols/r1/subjects/R1617S/experiments/catFR...,protocols/r1/subjects/R1617S/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1617S/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1617S/localizations/0/m...,NaN,6,R1617S,R1617S,3.4,protocols/r1/subjects/R1617S/experiments/catFR...


In [7]:
# load df from hamming run so can take only trials that loaded from it

hamming_df = pd.read_csv('/home1/john/SWR/compiled/catFR_hamming_HPC_df.csv') #hamming_HPC_df
hamming_df = hamming_df.drop_duplicates().reset_index(drop=True)
# change session value to int of the session number
for tr in range(len(hamming_df)):
    hamming_df.at[tr,'session'] = int(hamming_df.iloc[tr].session[7:])
hamming_df

,Unnamed: 0,session,subject,channel_num
0,0,0,R1004D,53
1,1,0,R1004D,53
2,2,0,R1004D,53
3,3,0,R1004D,53
4,4,0,R1004D,53
...,...,...,...,...
47060,47060,6,R1525J,190
47061,47061,6,R1525J,190
47062,47062,6,R1525J,190
47063,47063,6,R1525J,190


In [8]:
def getBestRegionNames(contact_pairs,contact_localization):

    # go through this manual list order and select the region from the best atlas available
    # details on list here: https://memory-int.psych.upenn.edu/InternalWiki/index.php/RAM_data
    
    # input: row of df from pairs and row of df from localization (or [] if localization doesn't exist)
    # output: the region and atlas for this contact (single electrode)
    
    region = []
    atlas_type = []
    region_reached = 0

    if 'stein.region' in contact_pairs: # if 'stein' in contact_row.dtype.names:
        test_region = str(contact_pairs['stein.region'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan') and (region_reached==0):
            region = test_region.lower()
            atlas_type = 'stein'
            region_reached = 1

    if 'das.region' in contact_pairs:
        test_region = str(contact_pairs['stein.region'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan') and (region_reached==0):
            region = test_region.lower()
            atlas_type = 'das'
            region_reached = 1
    if 'atlases.mtl' in contact_localization:    
        test_region = str(contact_localization['atlases.mtl'].values[0])    
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan') and (region_reached==0):
            region = test_region.lower()
            atlas_type = 'MTL_localization'
            region_reached = 1
            
    if 'atlases.whole_brain' in contact_localization:
        test_region = str(contact_localization['atlases.whole_brain'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan') and (region_reached==0):
            region = test_region.lower()
            atlas_type = 'wb_localization' 
            region_reached = 1
            
    if 'wb.region' in contact_pairs:
        test_region = str(contact_pairs['wb.region'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan') and (region_reached==0):
            region = test_region.lower()
            atlas_type = 'wb'  
            region_reached = 1
            
    if 'atlases.dk' in contact_localization:
        test_region = str(contact_localization['atlases.dk'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan') and (region_reached==0):
            region = test_region.lower()
            atlas_type = 'dk_localization'  
            region_reached = 1

    if 'dk.region' in contact_pairs:
        test_region = str(contact_pairs['dk.region'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan') and (region_reached==0):
            region = test_region.lower()
            atlas_type = 'dk'   
            region_reached = 1

    if 'ind.corrected.region' in contact_pairs:
        test_region = str(contact_pairs['ind.corrected.region'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan') and (region_reached==0):
            region = test_region.lower()
            atlas_type = 'ind.corrected' 
            region_reached = 1
            
    if 'ind.region' in contact_pairs:
        test_region = str(contact_pairs['ind.region'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan') and (region_reached==0):
            region = test_region.lower()
            atlas_type = 'ind'  
            region_reached = 1
    elif len(region) == 0:
        region = 'No atlas'
        atlas_type = 'No atlas'
                
    return region, atlas_type

In [9]:
ca1_contacts = []
ca1_atlases = []
ca1_idxs = [] # so can select these channels later
dg_contacts = []
dg_atlases = []
dg_idxs = [] # so can select these channels later


for row in exp_df.itertuples(): #sub_df.itertuples():   
    
    if sum( (row.subject==hamming_df.subject) & (row.session==hamming_df.session) )>0: # if this session is kept
        
        # get the HPC channels to target
        analyzed_channels = hamming_df[(row.subject==hamming_df.subject) & (row.session==hamming_df.session)].channel_num.values
    
        sub = row.subject; session = row.session; exp = row.experiment
        mont = int(row.montage); loc = int(row.localization)
        reader = CMLReadDFRow(row)
            
        pairs = reader.load('pairs')
        try:
            localizations = reader.load('localization')
#             localization_exists = 1
        except:
            localizations = []
        
        tal_struct, bipolar_pairs, mpchans = get_bp_tal_struct(sub, montage=mont, localization=loc)
        elec_regions,atlas_type,pair_number,has_stein_das = get_elec_regions(localizations,pairs)

        # for each pair, get the two contacts, and then figure out what their region is. Then save both in a list        
        contacts = reader.load('contacts')
        
        for channel in analyzed_channels:

            pair_label = pairs.iloc[channel].label
            split_pair = pair_label.split('-')
            
            region_pair = []
            atlas_pair = []
            contact_localization = []
            for contact in split_pair:
                contact_pairs = contacts.iloc[findInd(contacts.label==contact)]
                if len(localizations)>0:
                    contact_localization = localizations.iloc[findInd(localizations.iloc[:,0]['contacts'].index.values==contact)]

                region,atlas = getBestRegionNames(contact_pairs,contact_localization)

                region_pair.append(region) # get the pair of elecs for this bipolar pair
                atlas_pair.append(atlas)
        
                            
            if elec_regions[channel].find('ca1')>-1:
                ca1_contacts.append(region_pair)
                ca1_atlases.append(atlas_pair) 
                ca1_idxs.append(hamming_df[(row.subject==hamming_df.subject) & (row.session==hamming_df.session) & (channel==hamming_df.channel_num)].index.values[0])
            elif elec_regions[channel].find('dg')>-1:
                dg_contacts.append(region_pair)
                dg_atlases.append(atlas_pair)                
                dg_idxs.append(hamming_df[(row.subject==hamming_df.subject) & (row.session==hamming_df.session) & (channel==hamming_df.channel_num)].index.values[0])


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1004D/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1004D/tal/R1004D_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1004D/localizations/0/montages/0/neuroradiology/current_processed/contacts.json
/data/eeg/R1004D/tal/R1004D_talLocs_database_monopol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.p

In [10]:
ca1_contact_array = np.array(ca1_contacts)
for i_pair in range(len(ca1_contact_array)):
    for i_contact,contact in enumerate(ca1_contact_array[i_pair]):
        if len(contact)==0:
            ca1_contact_array[i_pair,i_contact] = ''
dg_contact_array = np.array(dg_contacts)
for i_pair in range(len(dg_contact_array)):
    for i_contact,contact in enumerate(dg_contact_array[i_pair]):
        if len(contact)==0:
            dg_contact_array[i_pair,i_contact] = ''            

ca1_elec_counts = np.unique(ca1_contact_array,return_counts=True)
dg_elec_counts = np.unique(dg_contact_array,return_counts=True)

len(ca1_contact_array)
ca1_elec_counts
sum(ca1_elec_counts[1])
len(dg_contact_array)
sum(dg_elec_counts[1])

ca1_as_rows = [[ca1_elec_counts[0][i],ca1_elec_counts[1][i]] for i in range(len(ca1_elec_counts[0]))]
dg_as_rows = [[dg_elec_counts[0][i],dg_elec_counts[1][i]] for i in range(len(dg_elec_counts[0]))]

# # save the count data
# np.savetxt("/home1/john/SWR/compiled/ca1_pair_electrode_counts.csv", 
#            ca1_as_rows,
#            delimiter=",",fmt='%s')
# np.savetxt("/home1/john/SWR/compiled/dg_pair_electrode_counts.csv", 
#            dg_as_rows,
#            delimiter=",",fmt='%s')

19621

(array(['', '"ba36"', '"ca1"', '"dg"', '"misc"', '"phc"', '"sub"',
        'fusiform', 'inferiortemporal', 'insula', 'left ca1', 'left ca3',
        'left cerebral white matter', 'left dg', 'left hippocampus',
        'left inf lat vent', 'left mtl wm', 'left phc',
        'left phg parahippocampal gyrus', 'left prc', 'left sub',
        'right amy', 'right ca1', 'right cerebral white matter',
        'right dg', 'right fug fusiform gyrus', 'right hippocampus',
        'right inf lat vent', 'right mtl wm',
        'right phg parahippocampal gyrus', 'right sub'], dtype=object),
 array([   31,   210,  7625,  1828,   443,    61,   321,   153,   153,
          157, 10658,   242,  1159,  1344,  1729,   100,   199,   208,
           36,    87,   451,   133,  6581,  1728,  1455,   287,   476,
          347,   529,    29,   482]))

39242

11251

22502

In [11]:
# how many CA1 and DG contacts have *ONLY* CA1 and DG surrounding them?
ca1_only_pairs = []
dg_only_pairs = []
for i_cp,contact_pair in enumerate(ca1_contacts):
    if ( (str(contact_pair[0]).find('ca1')>-1) & (str(contact_pair[1]).find('ca1')>-1) ):
        ca1_only_pairs.append(i_cp)
for i_cp,contact_pair in enumerate(dg_contacts):
    if ( (str(contact_pair[0]).find('dg')>-1) & (str(contact_pair[1]).find('dg')>-1) ):
        dg_only_pairs.append(i_cp)       
print('% of CA1 pairs coming from 2 CA1 electrodes: '+str(len(ca1_only_pairs)/len(ca1_contacts)*100))
print('% of DG pairs coming from 2 DG electrodes: '+str(len(dg_only_pairs)/len(dg_contacts)*100))

% of CA1 pairs coming from 2 CA1 electrodes: 39.90622292441772
% of DG pairs coming from 2 DG electrodes: 48.74233401475425


In [12]:
ca1_ca1_pair_idxs = np.array(ca1_idxs)[ca1_only_pairs]
dg_dg_pair_idxs = np.array(dg_idxs)[dg_only_pairs]
ca1_ca1_pair_df = hamming_df.iloc[ca1_ca1_pair_idxs].drop_duplicates()
ca1_ca1_pair_df
dg_dg_pair_df = hamming_df.iloc[dg_dg_pair_idxs].drop_duplicates()
dg_dg_pair_df
ca1_ca1_pair_df.to_csv('/home1/john/SWR/compiled/catFR_ca1_ca1_pair_df.csv') 
dg_dg_pair_df.to_csv('/home1/john/SWR/compiled/catFR_dg_dg_pair_df.csv') 

,Unnamed: 0,session,subject,channel_num
52,52,0,R1015J,11
64,64,0,R1015J,12
340,340,0,R1032D,12
361,361,0,R1032D,13
382,382,0,R1032D,14
...,...,...,...,...
46645,46645,6,R1525J,78
46675,46675,6,R1525J,94
46705,46705,6,R1525J,95
46735,46735,6,R1525J,134


,Unnamed: 0,session,subject,channel_num
10,10,0,R1004D,54
20,20,0,R1004D,55
2710,2710,0,R1065J,41
2818,2818,0,R1065J,42
3358,3358,0,R1065J,53
...,...,...,...,...
46381,46381,5,R1525J,147
46453,46453,5,R1525J,188
46855,46855,6,R1525J,146
46885,46885,6,R1525J,147
